<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/Others/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 크롤링

In [ ]:
#pip install beautifulsoup4

In [ ]:
import requests
import random
import time
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import tensorflow as tf
import numpy as np
import math

In [ ]:
base_url = 'https://movie.naver.com/movie/point/af/list.naver?&page={}'

data = []
quantity = 10000

start = int(time.time())

for page in range(1, quantity+1):
  url = base_url.format(page)
  res = requests.get(url)

  if(res.status_code == 200):
    soup = BeautifulSoup(res.text, 'lxml')
    reviews = soup.find_all(class_='title')
    for review in reviews:
      scr = int(review.find('em').text.strip())
      cmt = (review.select_one('br').next_sibling.strip())
      data.append([scr, cmt, len(cmt)])
    interval = round(random.uniform(0.2, 1.2), 2)
    time.sleep(interval)

  now = int(time.time())
  if((now - start)%10 == 0):
    for i in range(10):
      if i < ((page/quantity)*10)-1:
        print('■', end='')
      else:
        print('□', end='')
    if now - start < 60:
      print('...{:.2f}% {}sec'.format(((page/quantity)*100), (now - start)))
    elif now - start < 3600:
      print('...{:.2f}% {}min {}sec'.format(((page/quantity)*100), ((now - start)//60), ((now - start)%60)))
    else:
      print('...{:.2f}% {}hour {}min {}sec'.format(((page/quantity)*100), ((now - start)//3600), (((now - start)%3600)//60), (((now - start)%3600)%60)))

for i in range(10):
  print('■', end='')
print('...done! {}hour {}min {}sec'.format(((now - start)//3600), (((now - start)%3600)//60), (((now - start)%3600)%60)))

□□□□□□□□□□...0.08% 10sec
□□□□□□□□□□...0.22% 30sec
□□□□□□□□□□...0.30% 40sec
□□□□□□□□□□...0.37% 50sec
□□□□□□□□□□...0.53% 1min 10sec
□□□□□□□□□□...0.75% 1min 40sec
□□□□□□□□□□...0.82% 1min 50sec
□□□□□□□□□□...0.89% 2min 0sec
□□□□□□□□□□...0.96% 2min 10sec
□□□□□□□□□□...1.03% 2min 20sec
□□□□□□□□□□...1.10% 2min 30sec
□□□□□□□□□□...1.17% 2min 40sec
□□□□□□□□□□...1.18% 2min 40sec
□□□□□□□□□□...1.25% 2min 50sec
□□□□□□□□□□...1.40% 3min 10sec
□□□□□□□□□□...1.54% 3min 30sec
□□□□□□□□□□...1.68% 3min 50sec
□□□□□□□□□□...1.82% 4min 10sec
□□□□□□□□□□...1.97% 4min 30sec
□□□□□□□□□□...2.04% 4min 40sec
□□□□□□□□□□...2.12% 4min 50sec
□□□□□□□□□□...2.20% 5min 0sec
□□□□□□□□□□...2.27% 5min 10sec
□□□□□□□□□□...2.41% 5min 30sec
□□□□□□□□□□...2.47% 5min 40sec
□□□□□□□□□□...2.76% 6min 20sec
□□□□□□□□□□...2.83% 6min 30sec
□□□□□□□□□□...2.98% 6min 50sec
□□□□□□□□□□...3.05% 7min 0sec
□□□□□□□□□□...3.14% 7min 10sec
□□□□□□□□□□...3.20% 7min 20sec
□□□□□□□□□□...3.28% 7min 30sec
□□□□□□□□□□...3.36% 7min 40sec
□□□□□□□□□□...3.43% 7min 50sec
□□□

# 데이터 정리

## 전처리

In [ ]:
use_data = data.copy()

In [ ]:
for i in range(len(use_data)):
  use_data[i][1] = use_data[i][1].split(' ')

In [ ]:
label = {}
size = {}
count = 0

for i in use_data:
  for j in i[1]:
    if j not in label:
      label[j] = [count]
      size[j] = 1
      count += 1
    else:
      size[j] += 1

In [ ]:
for i in range(len(use_data)):
  tmp = []
  for j in label.keys():
    write = False
    for k in use_data[i][1]:
      if j == k and write == False:
        tmp.append(1)
        write = True
    if write == False:
      tmp.append(0)
  for k in tmp:
    use_data[i].append(k)

In [ ]:
for i in range(len(use_data)):
  del use_data[i][1]

In [ ]:
for i in reversed(list(size.keys())):
  if size.get(i) < 2:
    for j in range(len(use_data)):
      del use_data[j][label[i][0]+2]
    del label[i]

In [ ]:
column_name_list = list(label.keys())
column_name_list.insert(0, 'score')
column_name_list.insert(1, 'length')

['score', 'length', '대사', '잘', '빼곤', '잘봤어요', '액션도', '시원시원하고', '좋아요', '봤어요.', '정말', '대단한', '영화다', '', '출연', '해서', '영어', '좀', '했다고', '이', '영화에', '또', '데이비드', '출연료', '보니', '그리', '유명한', '감독도', '아니고', '내용', '참으로', '그지', 'ㅋㅋ', '현실적이라', '더', '없어도', '재밌을수', '눈빛이', '너무', '멋짐', '생각보다', '코믹하고', '재미있음', '글고', '박세완이라는', '새로운', '배우의', '코로나', '시국에', '즐겁게', '보았던', '영화.', '내', '공조2', '재밌었어요', '^^', '2시간', '않은', '런닝타임이', '길게', '뭐', '내가', '다른', '간만에', '느끼게', '만드는', '영화였다!!', 'ㅎㅎ', '보는', '동안', '지루함', '없이', '긴장하며', '봤다..', '좋은', '기대', '할때는', '뭔', '영화가', '재미', '코믹', '영화로서', '기발한', '웃음은', '영화로', '다크나이트', '이전에', '있었다', '힘든', '시간을', '김호중님의', '진심이', '담긴', '아름다운', '영화입니다.', 'ㅎㅎㅎ', '미쳤다', '근데', '문재앙', '언제', '담백하게', '하는', '아쉬움', '예전에', '공조1편', '보다', '이번에', '본', '2편이', '재미있었네요.', '처음부터', '끝까지', '시간가는줄', '모르고', '올해', '영화중', '최고입니다.', '굿~~~', '액션영화가', '이렇게', '예술로', '나를', '영화보고', '진짜', '많이', '본인', '얘기', '재밌어요', '영화관', '본게', '아쉬웠어요', '싫어서', '나왔으면', '영화', '많은거', '같은데', '이것도', '영화관에서', '했으면', '좋겠어요', '되면', '가서', '계속', '제

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 데이터프레임

In [ ]:
df = pd.DataFrame(use_data)
df.columns = list(column_name_list)
df.fillna(0, inplace=True)

In [ ]:
df.head()

,score,length,대사,잘,빼곤,잘봤어요,액션도,시원시원하고,좋아요,봤어요.,...,담으려고,지나온,평론가의,마땅한,친절히,기다릴,"톰켓,","탑건1,",톰켓,안되긴
0,7,33,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,10,24,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,10,17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,157,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 9700 entries, score to 안되긴
dtypes: int64(9700)
memory usage: 740.1 MB


In [ ]:
df.describe()

,score,length,대사,잘,빼곤,잘봤어요,액션도,시원시원하고,좋아요,봤어요.,...,담으려고,지나온,평론가의,마땅한,친절히,기다릴,"톰켓,","탑건1,",톰켓,안되긴
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.00000,10000.000000,10000.000000,...,10000.0000,10000.000000,10000.0000,10000.000000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000
mean,7.760900,47.822000,0.003000,0.047400,0.000600,0.00160,0.007300,0.00070,0.004700,0.003900,...,0.0001,0.000200,0.0001,0.000200,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
std,3.108391,59.896508,0.054693,0.212504,0.024489,0.03997,0.085132,0.02645,0.068399,0.062331,...,0.0100,0.014141,0.0100,0.014141,0.0100,0.0100,0.0100,0.0100,0.0100,0.0100
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0000,0.000000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,6.000000,17.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0000,0.000000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
50%,10.000000,31.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0000,0.000000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
75%,10.000000,58.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0000,0.000000,0.0000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
max,10.000000,1000.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000,...,1.0000,1.000000,1.0000,1.000000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [ ]:
df.to_csv('review.csv')

#분석

In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 5.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import category_encoders as ce
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_on_gpu = torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/hobby/네이버 영화 리뷰/review.csv')

In [ ]:
df.drop(df.columns[0], axis=1, inplace=True)

In [ ]:
df.head(1)

,score,length,대사,잘,빼곤,잘봤어요,액션도,시원시원하고,좋아요,봤어요.,...,담으려고,지나온,평론가의,마땅한,친절히,기다릴,"톰켓,","탑건1,",톰켓,안되긴
0,7,33,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = df.drop(['score'], axis=1)
y = df['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']
numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']

encoder = ce.OneHotEncoder(cols=list(X.columns))
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

rfc_100 = RandomForestClassifier(n_estimators=100, random_state=0)
rfc_100.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

TypeError: ignored

In [ ]:
y_pred_100 = rfc_100.predict(X_test)
print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred_100)))

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)
feature_scores = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature_scores[:10]

# 테스트

In [ ]:
test_string = '이 영화 정말 재미있어요'
test_data = []

for i in range(len(test_string)):
  test_data.append(test_string.split(' '))

tmp = []
for j in label.keys():
  write = False
  for k in test_data:
    if j == k and write == False:
      tmp.append(1)
      write = True
  if write == False:
    tmp.append(0)

print(tmp)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
model.predict(test_string)